#CNN 모델 직접 구현
###김태영
https://tykimos.github.io/2017/03/08/CNN_Getting_Started/

[순서] 

문제 정의하기 

데이터 준비하기 

데이터셋 생성하기 

모델 구성하기 

모델 학습과정 설정하기 

모델 학습시키기 

모델 평가하기 

###1 문제정의

In [ ]:
# 다중 클래스 분류
# 입력 -> 손으로 그린 삼각형, 사각형, 원 이미지
# 출력 -> 삼각형, 사각형, 원일 확률 나타내는 벡터

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기
np.random.seed(3) #매 시행마다 결과 달라지지 않도록

그림판 그림 

http://tykimos.github.io/warehouse/2017-3-8_CNN_Getting_Started_handwriting_shape.zip

###2 데이터셋 생성

케라스에서는 이미지 파일을 쉽게 학습시킬 수 있도록 ImageDataGenerator 클래스를 제공합니다. ImageDataGenerator 클래스는 데이터 증강 (data augmentation)을 위해 막강한 기능을 제공하는 데, 이 기능들은 다른 강좌에세 다루기로 하고, 본 강좌에서는 특정 폴더에 이미지를 분류 해놓았을 때 이를 학습시키기 위한 데이터셋으로 만들어주는 기능을 사용해보겠습니다.

먼저 ImageDataGenerator 클래스를 이용하여 객체를 생성한 뒤 flow_from_directory() 함수를 호출하여 제네레이터(generator)를 생성합니다. flow_from_directory() 함수의 주요인자는 다음과 같습니다.

첫번재 인자 : 이미지 경로를 지정합니다.
target_size : 패치 이미지 크기를 지정합니다. 폴더에 있는 원본 이미지 크기가 다르더라도 target_size에 지정된 크기로 자동 조절됩니다.
batch_size : 배치 크기를 지정합니다.
class_mode : 분류 방식에 대해서 지정합니다.
categorical : 2D one-hot 부호화된 라벨이 반환됩니다.
binary : 1D 이진 라벨이 반환됩니다.
sparse : 1D 정수 라벨이 반환됩니다.
None : 라벨이 반환되지 않습니다.
본 예제에서는 패치 이미지 크기를 24 x 24로 하였으니 target_size도 (24, 24)로 셋팅하였습니다. 훈련 데이터 수가 클래스당 15개이니 배치 크기를 3으로 지정하여 총 5번 배치를 수행하면 하나의 epoch가 수행될 수 있도록 하였습니다. 다중 클래스 문제이므로 class_mode는 ‘categorical’로 지정하였습니다. 그리고 제네레이터는 훈련용과 검증용으로 두 개를 만들었습니다.

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

In [ ]:


train_data = ImageDataGenerator(rescale=1./255)

train_generator = train_data.flow_from_directory(
        '/content/gdrive/MyDrive/kaggle/handwriting_shape/train',
        target_size=(24, 24),
        batch_size=10,
        class_mode='categorical')

test_data = ImageDataGenerator(rescale=1./255)

test_generator = test_data.flow_from_directory(
        '/content/gdrive/MyDrive/kaggle/handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=10,
        class_mode='categorical')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###3 모델 구성

영상 분류에 높은 성능을 보이고 있는 컨볼루션 신경망 모델을 구성해보겠습니다. 각 레이어들은 이전 강좌에서 살펴보았으므로 크게 어려움없이 구성할 수 있습니다.

컨볼루션 레이어 : 입력 이미지 크기 24 x 24, 입력 이미지 채널 3개, 필터 크기 3 x 3, 필터 수 32개, 활성화 함수 ‘relu’
컨볼루션 레이어 : 필터 크기 3 x 3, 필터 수 64개, 활성화 함수 ‘relu’
맥스풀링 레이어 : 풀 크기 2 x 2
플래튼 레이어
댄스 레이어 : 출력 뉴런 수 128개, 활성화 함수 ‘relu’
댄스 레이어 : 출력 뉴런 수 3개, 활성화 함수 ‘softmax’

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

###4 모델 학습과정 설정

모델을 정의했다면 모델을 손실함수와 최적화 알고리즘으로 엮어봅니다.

loss : 현재 가중치 세트를 평가하는 데 사용한 손실 함수 입니다. 다중 클래스 문제이므로 ‘categorical_crossentropy’으로 지정합니다.
optimizer : 최적의 가중치를 검색하는 데 사용되는 최적화 알고리즘으로 효율적인 경사 하강법 알고리즘 중 하나인 ‘adam’을 사용합니다.
metrics : 평가 척도를 나타내며 분류 문제에서는 일반적으로 ‘accuracy’으로 지정합니다.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

###5 모델 학습시키기

케라스에서는 모델을 학습시킬 때 주로 fit() 함수를 사용하지만 제네레이터로 생성된 배치로 학습시킬 경우에는 fit_generator() 함수를 사용합니다. 본 예제에서는 ImageDataGenerator라는 제네레이터로 이미지를 담고 있는 배치로 학습시키기 때문에 fit_generator() 함수를 사용하겠습니다.

첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 train_generator으로 지정합니다.
steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정합니다. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정합니다.
epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정합니다. 100번을 반복적으로 학습시켜 보겠습니다.
validation_data : 검증데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 validation_generator으로 지정합니다.
validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정합니다. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정합니다.

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=5,
        epochs=100,
        validation_data=test_generator,
        validation_steps=5)

###6 모델 평가

학습한 모델을 평가해봅니다. 제네레이터에서 제공되는 샘플로 평가할 때는 evaluate_generator 함수를 사용합니다.

In [ ]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

###7 모델 사용

모델 사용 시에 제네레이터에서 제공되는 샘플을 입력할 때는 predict_generator 함수를 사용합니다. 예측 결과는 클래스별 확률 벡터로 출력되며, 클래스에 해당하는 열을 알기 위해서는 제네레이터의 ‘class_indices’를 출력하면 해당 열의 클래스명을 알려줍니다.

In [ ]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

#전체 코드

In [ ]:
#GPU 연결 지속 코드 + F12에서

function ClickConnect(){ 
console.log("Working");  
document.querySelector("colab-toolbar-button#connect").click()  
} 
setInterval(ClickConnect,60000)


In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

In [ ]:
# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pylab as plt

image_gen = ImageDataGenerator(rescale=(1/255.))

train_gen = image_gen.flow_from_directory('/content/gdrive/MyDrive/kaggle/food_minidata/train',
                                          batch_size = 32,
                                          target_size = (224, 224,),
                                          classes=['apple_pie', 'bibimbap'],
                                          class_mode = 'binary',
                                          seed = 2020)
test_gen = image_gen.flow_from_directory('/content/gdrive/MyDrive/kaggle/food_minidata/test',
                                          batch_size = 32,
                                          target_size = (224, 224,),
                                          classes=['apple_pie', 'bibimbap'],
                                          class_mode = 'binary',
                                          seed = 2020)

class_labels = ['apple_pie', 'bibimbap']
batch = next(train_gen)
images, labels = batch[0], batch[1] # 0번 이미지데이터 1번 레이블
print(labels[:10])
plt.figure(figsize=(16,8))
for i in range(32) :
    ax = plt.subplot(4,8,i+1)
    plt.imshow(images[i])
    plt.title(class_labels[labels[i].astype(np.int)])
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# 0. 사용할 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# 1. 데이터 생성하기
img = [24,24,3]

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/gdrive/MyDrive/kaggle/food_1-1000/train300',
        target_size=(24,24),
        batch_size=10,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/gdrive/MyDrive/kaggle/food_1-1000/test300',
        target_size=(24,24),    
        batch_size=10,
        class_mode='categorical')

# 상수 define -> 이미지 크기 조정

# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# # 저장한 모델 불러오기
# from keras.models import load_model
# model = load_model('/content/gdrive/MyDrive/my_model.h5')


# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



# 4. 모델 학습시키기
model.fit(
        train_generator,
        steps_per_epoch=63,
        epochs=50,
        validation_data=test_generator,
        validation_steps=27)

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output) 
print(test_generator.filenames,"\n")

# keras 모델 저장
from keras.models import load_model
model.save('/content/gdrive/MyDrive/my_model.h5')

In [ ]:
# 현재 작업중인 디렉토리 출력
!pwd

# 현재 작업하는 위치 변경
%cd gdrive/'My Drive'

# 파일 생성
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')

In [ ]:
# keras 모델 저장, 불러오기
from keras.models import load_model
model.save('my_model.h5')

In [ ]:
from keras.models import load_model
model = load_model('/content/gdrive/MyDrive/my_model.h5')

https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/keras/save_and_load.ipynb?hl=ko#scrollTo=SkGwf-50zLNn 

딥러닝 모델 저장 및 불러오기

In [ ]:
import tensorflow as tf

# 모델 구조를 확인합니다
model.summary()

In [ ]:
# 복원된 모델을 평가합니다
loss, acc = model.evaluate(test_generator, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(model.predict(test_generator).shape)

In [ ]:
# 이미지 파일 사진 크기 변경

import os
import glob
from PIL import Image

files = glob.glob('/content/gdrive/MyDrive/kaggle/food_minidata/test/*.jpg')

for f in files:
    img = Image.open(f)
    img_resize = img.resize((int(24), int(24)))
    title, ext = os.path.splitext(f)
    img_resize.save(title + '_half' + ext)